In [35]:
import pandas as pd
import numpy as np

In [36]:
# list fields, just for information

list_fields = pd.read_json('https://www.globaltradetracker.com/api/rest/getreport/fields?token=C0610496D526FC06563CBFF90EE6C5D8&lang=en')
#list_fields

#### Example of an API: 

https://www.globaltradetracker.com:443/api/rest/getreport?token=C0610496D526FC06563CBFF90EE6C5D8&reporter=CH&from=2021-01&hscode=280910&to=&latestmonths=&impexp=I&source=DEFAULT

1. reporter => get the list
2. from => 2019-01
3. hscode (product) => get the list
4. to => let it empty 
5. impexp (trade flow) => import, export
6. source: DEFAULT, EURPSTAT, COMTRADE



##### 1. list of reporter

In [37]:
# 1. list GTT countries

list_countries = pd.read_json('https://www.globaltradetracker.com/api/rest/getreport/countries?token=C0610496D526FC06563CBFF90EE6C5D8&lang=en')
list_countries = pd.json_normalize(list_countries['country'])
list_countries

,alphageonom2,name,reportingcountry,source,update_frequency,lastavailable_date,data_lastreceived,firstavailable_date,reportercode,description,has_transport,has_subdivisions,has_ports
0,00,Neutral Zone,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,"Areas, nes",false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,03,Special Categories,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,04,US Misc. Pacific Isds,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,05,Bunkers,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,YT,Mayotte,true,UN Comtrade,Y,2009,2018-11-21 09:16:37 GMT,2000,NaN,NaN,NaN,NaN,NaN
333,YU,Yugoslavia,false,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
334,ZA,South Africa,true,South African Revenue Service,M,202111,2021-12-28 11:09:46 GMT,200201,NaN,NaN,true,NaN,true
335,ZM,Zambia,true,Central Statistical Office Zambia,M,202101,2021-03-09 10:05:03 GMT,200901,NaN,NaN,NaN,NaN,NaN


In [78]:
# shoud be in the alphageonom2 but reportingcountry==true

repo = list_countries.drop(list_countries[list_countries.reportingcountry == 'false'].index)
repo.head()

,alphageonom2,name,reportingcountry,source,update_frequency,lastavailable_date,data_lastreceived,firstavailable_date,reportercode,description,has_transport,has_subdivisions,has_ports
17,AD,Andorra,true,UN Comtrade,Y,2018,2019-05-06 12:44:49 GMT,1995,NaN,NaN,NaN,NaN,NaN
18,AE,United Arab Emirates,true,UN Comtrade,Y,2020,2021-11-11 08:53:50 GMT,1991,NaN,NaN,NaN,NaN,NaN
19,AF,Afghanistan,true,UN Comtrade,Y,2019,2021-11-03 08:40:53 GMT,2008,NaN,NaN,NaN,NaN,NaN
20,AG,Antigua and Barbuda,true,UN Comtrade,M,202103,2021-10-09 14:36:04 GMT,201001,NaN,NaN,NaN,NaN,NaN
21,AI,Anguilla,true,UN Comtrade,Y,2008,2018-11-21 09:16:37 GMT,2000,NaN,NaN,NaN,NaN,NaN


##### 2. from
lets get from 2019

##### 3. hscode
imported from Trade Flows-GTT Translator

In [39]:
hsCode = pd.read_csv('hsCode.csv')
hsCode.head()

,HS Code,Description,Short Descr.,Family,Sub-Family,Product,Unnamed: 6
0,251010,NATURAL CALCIUM PHOSPHATES AND NATURAL ALUMINI...,Unground Aluminium/Calcium Phosphates & Phosph...,P,Phosphate Rock,PR,NaN
1,251020,NATURAL CALCIUM PHOSPHATES AND NATURAL ALUMINI...,Ground Aluminium/Calcium Phosphates & Phosphat...,P,Phosphate Rock,PR,NaN
2,280920,"PHOSPHORIC ACID; POLYPHOSPHORIC ACIDS, WHETHER...",Phosphoric Acids,P,Phosphoric Acids,PA,NaN
3,280910,DIPHOSPHORUS PENTAOXIDE,Diphosphorus Ppentaoxide,P,Phosphoric Acids,PA,NaN
4,281410,ANHYDROUS AMMONIA,Anhydrous Ammonia,N,N Products,NH3,NaN


##### 4. to 
now

##### 5. impexp 
lets do it with import (I), export (E) 

##### 6. source 
DEFAULT, EURPSTAT, COMTRADE

In [64]:
PATTERN = 'https://www.globaltradetracker.com:443/api/rest/getreport?token=C0610496D526FC06563CBFF90EE6C5D8&reporter={}&from=2019-01&hscode={}&to=&latestmonths=&impexp={}&source={}'

def crawl_data(reporter, hscode, impexp, source):
    
    crawl = pd.read_json(PATTERN.format(reporter, hscode, impexp, source))
    
    return crawl


In [83]:
def combine_data():
    data = []
    
    reporter = repo['alphageonom2'].unique()
    hscode = hsCode['HS Code'].unique()
    impexp = ["I", "E"]
    source = ["DEFAULT", "EURPSTAT", "COMTRADE"]
    
    for (reporter, hscode, impexp, source) in zip(reporter, hscode, impexp, source):
        
        data.append(crawl_data(reporter, hscode, impexp, source))
        
    df = pd.concat(data)

    return df

In [84]:
df = combine_data()

HTTPError: HTTP Error 400: 400

In [82]:
def combine_data():
    data = []

    for reporter in repo['alphageonom2'].unique()
        hscode in hsCode['HS Code'].unique()
        impexp in ["I", "E"]
        source in ["DEFAULT", "EURPSTAT", "COMTRADE"]
            
        data.append(crawl_data(reporter, hscode, impexp, source))
        
    df = pd.concat(data)

    return df

SyntaxError: invalid syntax (1434343575.py, line 4)